<a href="https://colab.research.google.com/github/Bonorinoa/MapReduce_Python/blob/main/MapReduce_TexProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -f hadoop-3.3.2.tar.gz*

In [ ]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz

--2022-04-25 23:26:36--  https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638660563 (609M) [application/x-gzip]
Saving to: ‘hadoop-3.3.2.tar.gz’

hadoop-3.3.2.tar.gz 100%[===================>] 609.07M  1.75MB/s    in 6m 36s  

2022-04-25 23:33:13 (1.54 MB/s) - ‘hadoop-3.3.2.tar.gz’ saved [638660563/638660563]



In [ ]:
!tar -xzf hadoop-3.3.2.tar.gz

In [ ]:
#copy  hadoop file to user/local
!cp -r hadoop-3.3.2/ /usr/local/

In [ ]:
#To find the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


To set the java path, go to /usr/local/hadoop-3.3.2/etc/hadoop/hadoop-env.sh then

* Look for the line `export JAVA_HOME=`
* Replace it with `export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64/`
* Remove the comments (the notebook will automatically save the shell script)

In [ ]:
# Chek if Hadoop was correctly installed
!/usr/local/hadoop-3.3.2/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

In [ ]:
!pip install mrjob

     |████████████████████████████████| 439 kB 5.2 MB/s 


## The goal is to compute the statistical support for the input list of tuples.
### Problem:

Let I = {i1, i2, ..., in} be a set of n items.
Let D = {t1, t2, ..., tm} be a set of m transactions.

We must identify an association of the form Y → X, where X, Y ⊆ I, and X ∩ Y = 0.

In [ ]:
# Load Groceries.csv or tinyGroceries.csv
from google.colab import files
import pandas as pd

uploaded = files.upload()

Saving Groceries.csv to Groceries.csv


In [ ]:
%%file supportandsort.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re

from itertools import combinations
import pandas as pd

class supportandsort(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   reducer=self.count_pairs),
            MRStep(mapper=self.make_counts_key,
                   reducer = self.output_support)
        ]

    def mapper(self, _, row):
        
        items = row.split(',')

        combs = list(combinations(items, 2))

        for pair in combs:

          yield ( (pair, 1) )

    def count_pairs(self, pair, counts):
        yield  pair, sum(counts)

    def make_counts_key(self, pair, count):
        yield str(count).rjust(5,'0'), pair

    def output_support(self, count, pairs):
        for pair in pairs:
            yield count, pair

    # def reducer(self, key, value):
    #     yield ( key, sum(value) )


if __name__ == '__main__':
    supportandsort.run() 

Overwriting supportandsort.py


## First, let's get the logic of the code to count pairs of objects right with a smaller example

In [ ]:
# Change below the name of the Python/MrJob program and the file(s) used by it
# The datafiles can have extensions (e.g. .csv, .txt)  or be without extension.
!python supportandsort.py Groceries.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/supportandsort.root.20220425.233453.253039
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/supportandsort.root.20220425.233453.253039/output
Streaming final output from /tmp/supportandsort.root.20220425.233453.253039/output...
"00036"	["Ready made", "Fresh Vegetables"]
"00036"	["Alcohol", "Fresh Vegetables"]
"00040"	["Frozen foods", "Fresh Vegetables"]
"00040"	["Toiletries", "Tinned Goods"]
"00043"	["Fresh Vegetables", "Snacks"]
"00046"	["Fresh Vegetables", "Bakery goods"]
"00046"	["Ready made", "Toiletries"]
"00047"	["Alcohol", "Toiletries"]
"00050"	["Frozen foods", "Toiletries"]
"00054"	["Fresh Vegetables", "Tinned Goods"]
"00056"	["Bakery goods", "Toiletries"]
"00058"	["Toiletries", "Snacks"]
"00085"	["Frozen foods", "Milk"]
"00090"	["Alcohol", "Milk"]
"00098"	["Milk", "Snacks"]
"00100"	["Milk", "Tinned Goods"]
"00105"	["Ready made", "Milk"]
"0

## Let's run it in Hadoop now

In [ ]:
%%file ~/.mrjob.conf
runners:
    hadoop:
      hadoop_bin: /usr/local/hadoop-3.3.2/bin/hadoop
      hadoop_streaming_jar: /usr/local/hadoop-3.3.2/share/hadoop/tools/lib/hadoop-streaming-3.3.2.jar
      hadoop_tmp_dir: file:///content/tmp/mrjob

Writing /root/.mrjob.conf


In [ ]:
# Change below the name of the Python/MrJob program and the file(s) used by it
# Note that the option -r hadoop changes the execution  from local to the hadoop framework
!python supportandsort.py -r hadoop  Groceries.csv

Using configs in /root/.mrjob.conf
Using Hadoop version 3.3.2
Creating temp directory /tmp/supportandsort.root.20220425.233509.579054
uploading working dir files to file:///content/tmp/mrjob/supportandsort.root.20220425.233509.579054/files/wd...
Copying other local files to file:///content/tmp/mrjob/supportandsort.root.20220425.233509.579054/files/
Running step 1 of 2...
  Loaded properties from hadoop-metrics2.properties
  Scheduled Metric snapshot period at 10 second(s).
  JobTracker metrics system started
  JobTracker metrics system already initialized!
  Total input files to process : 1
  number of splits:1
  Submitting tokens for job: job_local1548679558_0001
  Executing with tokens: []
  Creating symlink: /tmp/hadoop-root/mapred/local/job_local1548679558_0001_1385ceb9-2f4e-4c22-a292-5d406c2dba14/mrjob.zip <- /content/mrjob.zip
  Localized file:/content/tmp/mrjob/supportandsort.root.20220425.233509.579054/files/wd/mrjob.zip as file:/tmp/hadoop-root/mapred/local/job_local1548679558